<a href="https://colab.research.google.com/github/PhucDuy/tonga_week_5_assignment/blob/master/%5Bphuc_tran%5DMovie_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Movie Reviews Prediction

## Logistic regression with Sklearn

### Import data and library

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Import pandas, numpy and the dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [74]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [75]:
!pip install contractions
!pip install autocorrect


### Read data and clean up

In [0]:
# Read file 
data = pd.read_csv('/content/drive/My Drive/FTMLE - Tonga/Data/movie_review.csv', encoding='utf-8', sep='\t')
data = data

In [77]:
data.head(10)

,id,review,sentiment
0,5814_8,With all this stuff going down at the moment w...,1
1,2381_9,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,7759_3,The film starts with a manager (Nicholas Bell)...,0
3,3630_4,It must be assumed that those who praised this...,0
4,9495_8,Superbly trashy and wondrously unpretentious 8...,1
5,8196_8,I dont know why people think this is such a ba...,1
6,7166_2,"This movie could have been very good, but come...",0
7,319_1,"A friend of mine bought this film for Â£1, and...",0
8,8713_10,<br /><br />This movie is full of references. ...,1
9,2486_3,"What happens when an army of wetbacks, towelhe...",0


In [0]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', -1)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')


In [0]:
# Drop id
data.drop(labels=['id'], axis=1, inplace=True)

In [80]:
test_text = data.iloc[data['review'].str.len().idxmax()].iloc[0]
print_full(test_text)

Match 1: Tag Team Table Match Bubba Ray and Spike Dudley vs Eddie Guerrero and Chris Benoit Bubba Ray and Spike Dudley started things off with a Tag Team Table Match against Eddie Guerrero and Chris Benoit. According to the rules of the match, both opponents have to go through tables in order to get the win. Benoit and Guerrero heated up early on by taking turns hammering first Spike and then Bubba Ray. A German suplex by Benoit to Bubba took the wind out of the Dudley brother. Spike tried to help his brother, but the referee restrained him while Benoit and Guerrero ganged up on him in the corner. With Benoit stomping away on Bubba, Guerrero set up a table outside. Spike dashed into the ring and somersaulted over the top rope onto Guerrero on the outside! After recovering and taking care of Spike, Guerrero slipped a table into the ring and helped the Wolverine set it up. The tandem then set up for a double superplex from the middle rope which would have put Bubba through the table, but

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


### Text Processing

In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

stop = stopwords.words('english')

#### Removing special characters and "trash"


In [82]:
!pip install beautifulsoup4
!pip install -U spacy
!python -m spacy download en_core_web_sm

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
import re
import contractions
import string
from bs4 import BeautifulSoup
import unicodedata

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove html tags
    text = strip_html_tags(text)
    # Remove accents 
    text = remove_accented_chars(text)
    # remove extra newlines
    text = re.sub(r'[\r|\n|\r\n]+', ' ',text)

    # Expanding contractions words
    text = contractions.fix(text)
    
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)

    # text.replace('...',' ')
    # text.replace('-',' ')

    # Remove accents 
    text = remove_accented_chars(text)

    # # insert spaces between special characters to isolate them    
    # special_char_pattern = re.compile(r'([{.(-)!}])')
    # text = remove_special_characters(text)  

    # # remove extra whitespace
    # text = re.sub(' +', ' ', text)

    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))

    return text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def strip_html_tags(text):
    soup = BeautifulSoup(text, "lxml")
    stripped_text = soup.get_text()
    return stripped_text

# # Removing Special Characters
def remove_special_characters(text):
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    return text


In [84]:
%%time
test = data['review'].sample(10)
print_full(test)
print_full(test.apply(preprocessor))

16674    This ensemble piece about adults who return to the formulative Summer camp of their youth was a very quiet entrant and exiter to the cinemas in the Autumn of 1993. I'd say that was a shame,but then again,some of the better movies ARE quiet releases that don't get much hype or praise.<br /><br />Diane LAne,Kevin Pollack,Elizabeth Perkins,Vincent Spano,Julie Warner,Bill Paxton,Kimberly Williams,Matt Craven and Alan Arkin(who is painfully good here) are the group of actors who flesh out these roles as people who have grown old with good,bad and funny memories of summers gone past. This film covers the gambit of emotions,mostly pleasant,and the film never hammers away at the viewer to feel what the characters are feeling,preferring to allow the viewer to enter into the memories on their own. Since I am a movie viewer who bristles at bluntly,brazen manipulation in films,this is something that I appreciate from writer/director Mike Binder.<br /><br />This film's a great cheap rent,a

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


#### Prepare tokenizer

In [0]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string
from sklearn.base import TransformerMixin
from nltk.tokenize.toktok import ToktokTokenizer


punctuations = string.punctuation
tokenizer = ToktokTokenizer()
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = spacy.load("en_core_web_sm")

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    sentence = remove_stopwords(sentence)
    # Keep meaningfull words
    # meaningfull_words = get_meaningfull_words(sentence)
    # for word in meaningfull_words:
    #   sentence = sentence.replace(word,'')
    # Creating our token object, which is used to create documents with linguistic annotations.
    tokens = parser(sentence)
    # # Lemmatizing each token and converting each token into lowercase
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens ]
    # tokens.extend(meaningfull_words)

   # return preprocessed list of tokens
    return tokens


def spacy_filter_meaningfull_word_tokenizer(sentence):
    sentence = remove_stopwords(sentence)
    # Keep meaningfull words
    meaningfull_words = get_meaningfull_words(sentence)
    for word in meaningfull_words:
      sentence = sentence.replace(word,'')
    # Creating our token object, which is used to create documents with linguistic annotations.
    tokens = parser(sentence)
    # # Lemmatizing each token and converting each token into lowercase
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens ]
    tokens.extend(meaningfull_words)

   # return preprocessed list of tokens
    return tokens

def get_meaningfull_words(sentence):
  parser = spacy.load("en_core_web_sm")
  tokens = parser(sentence)
  return [ent.text for ent in tokens.ents]



# # Removing Stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stop_words]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text



# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()


In [86]:
%%time
test = data['review'].sample(10)
print_full(test)
print_full(test.apply(preprocessor).apply(spacy_tokenizer))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


18154    A well-made run-of-the-mill movie with a tragic ending. Pluses: The way the story moves - begins with Soorya struggling to live followed by a long flashback about why he's there. The Music. A disinterested look at the life of policemen. Minuses: The violence and the gore, but I guess they add to the realistic effects. Still, having people's heads chopped off and sent in boxes and sacks could have been avoided.<br /><br />No complaints - 7/10                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

### Split train and test sample

In [0]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split
X = data['review']
y = data['sentiment']
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size=0.2)


In [88]:
print(X.shape)
print(y.shape)
print(X_train.shape)
print(y_train.shape)

(22500,)
(22500,)
(18000,)
(18000,)


### Classification



In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals.joblib import Memory
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer,preprocessor=preprocessor,
                               ngram_range=(1,1), max_df = 0.5, min_df = 1,sublinear_tf=True)

log_reg = LogisticRegression(n_jobs=-1,random_state=0,C=3.5938136638046276)
tfidf_pipe =  Pipeline([("cleaner", predictors()),
                 ('vect', tfidf_vector),
                 ('clf', log_reg)],memory = Memory(location='/content/cache'))


In [0]:
%%time
# Train model

# model generation
tfidf_pipe.fit(X_train,y_train)
# Predict

tfidf_pipe_predictions = tfidf_pipe.predict(X_test)
# Evaluate
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, tfidf_pipe_predictions))
print("Logistic Regression Precision:",metrics.precision_score(y_test, tfidf_pipe_predictions))
print("Logistic Regression Recall:",metrics.recall_score(y_test, tfidf_pipe_predictions))

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(<__main__.predictors object at 0x7f455ff2d080>, 20989    You do realize that you've been watching the E...
22481    To me, the final scene, in which Harris respon...
21321    Bo is Jane Parker, whose long-lost anthropolog...
6571     This wasn't really a very good movie. There we...
7716     She's not Michael Jordan<br /><br />Think of a...
                               ...                        
3477     This is a long lost horror gem starring Sydney...
6501     As with a bunch of guys at school we must give...
19040    Wow, praise IMDb and Google, for I have been t...
15058    This was the worst movie I've ever seen in the...
10012    A visit by Hitler in Rome is the backdrop of t...
Name: review, Length: 18000, dtype: object, 
20989    0
22481    1
21321    0
6571     0
7716     1
        ..
3477     1
6501     1
19040    1
150

In [0]:
tfidf_filter_vector = TfidfVectorizer(tokenizer = spacy_filter_meaningfull_word_tokenizer,preprocessor=preprocessor,
                               ngram_range=(1,1), max_df = 0.5, min_df = 1,sublinear_tf=True)

log_reg = LogisticRegression(n_jobs=-1,random_state=0,C=3.5938136638046276)
tfidf_filter_pipe =  Pipeline([("cleaner", predictors()),
                 ('vect', tfidf_filter_vector),
                 ('clf', log_reg)],memory = Memory(location='/content/cache'))


In [71]:
%%time
# Train model

# model generation
tfidf_filter_pipe.fit(X_train,y_train)
# Predict

tfidf_filter_pipe_predictions = tfidf_filter_pipe.predict(X_test)
# Evaluate
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, tfidf_filter_pipe_predictions))
print("Logistic Regression Precision:",metrics.precision_score(y_test, tfidf_filter_pipe_predictions))
print("Logistic Regression Recall:",metrics.recall_score(y_test, tfidf_filter_pipe_predictions))

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.5, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2',
                preprocessor=<function preprocessor at 0x7f4564919488>,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function spacy_filter_meaningfull_word_tokenizer at 0x7f4564031f28>,
                use_idf=True, vocabulary=None), 
[ 'channel surfing and caught this on logo. it was one of those \\i have to '
  'watch this because it\'s so horribly bad\\" moments, like roadhouse without '
  'the joy. the writing is atrocious; completely inane a

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:315: UserWarning: Persisting input arguments took 1.79s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])


Logistic Regression Accuracy: 0.845
Logistic Regression Precision: 0.8207547169811321
Logistic Regression Recall: 0.8787878787878788
CPU times: user 10min 44s, sys: 7.53 s, total: 10min 51s
Wall time: 10min 53s


In [0]:
# Create hyperparameter options
# Create regularization hyperparameter space
C = np.logspace(0,5,10)
ngram_range = [(1,1),(1,2),(1,3),(2,2),(2,3)]
max_df = (0.25,0.5,0.75,1)
min_df = (0.25,0.5,0.75,1)
binary = (True,False)
norm = ('l1','l2')
use_idf = (True,False)
smooth_idf = (True,False)
sublinear_tf = (True,False)

param_grid = {
  'clf__C': C,
  # 'vect__ngram_range': ngram_range,
  # 'vect__max_df': max_df,'vect__min_df': min_df
  # 'vect__binary': binary, 'vect__norm': norm,
  # 'vect__use_idf': use_idf, 'vect__smooth_idf': smooth_idf,
  # 'vect__sublinear_tf':  sublinear_tf
}
gs = GridSearchCV(tfidf_pipe, param_grid,n_jobs=-1 ,cv=5, verbose=1)


In [0]:
from sklearn.externals.joblib import parallel_backend

# model generation
# with parallel_backend('multiprocessing'):
#   gs.fit(X_train,y_train)

In [0]:
%%time
# Predicting with a test dataset
gs_predictions = gs.predict(X_test)

CPU times: user 24.3 s, sys: 100 ms, total: 24.4 s
Wall time: 24.5 s


In [0]:
from sklearn import metrics


# View best hyperparameters
print('Best params:', gs.best_params_)
# print('Best C:', gs.best_estimator_.get_params()['C'])
# print('Best dual:', gs.best_estimator_.get_params()['dual'])
# print('Best fit_intercept:', gs.best_estimator_.get_params()['fit_intercept'])
# print('Best class_weight:', gs.best_estimator_.get_params()['class_weight'])
# print('Best random_state:', gs.best_estimator_.get_params()['random_state'])
# print('Best solver:', gs.best_estimator_.get_params()['solver'])


# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, gs_predictions))
print("Logistic Regression Precision:",metrics.precision_score(y_test, gs_predictions))
print("Logistic Regression Recall:",metrics.recall_score(y_test, gs_predictions))


Best params: {'clf__C': 3.5938136638046276}
Logistic Regression Accuracy: 0.8805387205387205
Logistic Regression Precision: 0.8723854911305269
Logistic Regression Recall: 0.8905405405405405


In [0]:
tweets = [
    "This is really bad",
    "I love this!",
    ":)",
]

preds = clf.predict_proba(tweets)

for i in range(len(tweets)):
    print(f'{tweets[i]} --> Positive, Negative = {preds[i]}')

#### Save Model

In [58]:
import pickle
import os

pickle.dump(tfidf_pipe, open(os.path.join('models', 'movie_review.pkl'), 'wb'), protocol=4)

NameError: ignored

In [0]:
# Evaluate model
evaluation_data = pd.read_csv('/content/drive/My Drive/FTMLE - Tonga/Data/movie_review_evaluation.csv',sep='\t')
X_evaluation = evaluation_data['review']
y_pred = tfidf_pipe.predict(X_evaluation)
pred_sentiment = pd.DataFrame(y_pred,columns=['sentiment'])
evaluation_data = pd.concat([X_pred,pred_sentiment], axis=1)
evaluation_data.to_csv('/content/phuc.tran.csv')

### Addition